In [6]:
import requests
import pandas as pd
import unicodedata
from bs4 import BeautifulSoup 
import mysql.connector as mdb
import warnings
warnings.filterwarnings("ignore")
import os
from datetime import datetime
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 500)


In [7]:
# Definindo url e fazendo request
url = 'http://cbm.al.gov.br/paginas/ocorrencias/true'
resposta = requests.get(url)
page = resposta.text
soup = BeautifulSoup(page, 'html.parser')

In [14]:
# Conectando ao MySQL
con = mdb.connect(user ='seu usuário', password ='sua senha', database ='ocorrencias_bombeiros')

cursor = con.cursor()
def insert(table, data):
    query = "INSERT INTO " + table + " SET " 
    comb = []
    for field in data:
        if(field):
            comb.append("%s = '%s'" % (field, data[field]))
    query += ", ".join(comb) 
    cursor.execute(query) 


In [15]:
#definindo função para tirar acentos
def strip_accents(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

In [ ]:
#Criando tabela no MySQL
cursor.execute('create table ocorrencias (id MEDIUMINT NOT NULL AUTO_INCREMENT, atendimento TEXT NOT NULL, datahora TEXT NOT NULL, ocorrencia TEXT NOT NULL, local TEXT NOT NULL, viaturas TEXT NOT NULL, militares TEXT NOT NULL, dhcoleta TEXT NOT NULL, PRIMARY KEY (id));')
 

In [16]:
#Fazendo scraping e inserindo no MySQL
atendimento = soup.findAll('h4')
datahora = soup.findAll('p', attrs={'class':"list-group-item-text"})
ax = []
asd = {}

z = 0
for j in range(0, len(datahora)):          
    if z == 0:
           
        asd['datahora'] =  strip_accents(datahora[j].text.replace(u'\xa0', u'')).replace("'", "").replace(',', '.')
        z += 1  
    
    elif z == 1:
        asd['ocorrencia'] = strip_accents(datahora[j].text.replace(u'\xa0', u'')).replace("'", "").replace(',', '.')
        z += 1 
         
    elif z == 2:
        asd['local'] = strip_accents(datahora[j].text.replace(u'\xa0', u'')).replace("'", "").replace(',', '.')
        z += 1  
         
    elif z == 3:
        asd['viaturas'] = strip_accents(datahora[j].text.replace(u'\xa0', u'').replace("'", "").replace('.', '').replace('VIATURAS EMPREGADAS', ''))
        z += 1  
       
    elif z == 4:
        asd['militares'] = strip_accents(datahora[j].text.replace(u'\xa0', u'')).replace("'", "").replace('MILITARES EMPREGADOS.', '')
        asd['atendimento'] =  strip_accents(atendimento[int(j/5)].text.replace(u'\xa0', u''))
        asd['dhcoleta'] = datetime.now()
        ax.append(asd)
        insert('ocorrencias', asd)
        asd = {}
        z=0

In [17]:
#Retirando observações repetidas, fazendo o conmit e fechando o cursor.
cursor.execute("DELETE a FROM ocorrencias AS a, ocorrencias AS b WHERE a.datahora=b.datahora AND a.local=b.local AND a.id > b.id;")
cursor.execute("SELECT * FROM ocorrencias")
g = cursor.fetchall()
con.commit()
cursor.close()
con.close()